In [4]:
import random
import math
import hashlib
import base64

# Fungsi untuk mengecek apakah suatu bilangan merupakan bilangan prima
def is_prime(number):
    if number < 2:
        return False
    for i in range(2, number // 2 + 1):
        if number % i == 0:
            return False
    return True

def file_to_base64(path):
    with open(path, "rb") as path_file:
        # Baca File
        opened_file = path_file.read()

        # Encode file ke dalam format Base64
        result = base64.b64encode(opened_file).decode("utf-8")

    return result

# Fungsi untuk mengecek apakah suatu bilangan merupakan bilangan prima
def get_prime_input(message):
    while True:
        try:
            user_input = int(input(message))
            if is_prime(user_input):
                return user_input
            else:
                break
        except ValueError:
            print("Inputan tidak valid tolong masukan bilangan bulat.")
            break

# Fungsi untuk mencari nilai d (private key) dari nilai e (public key) dan totient n
def mod_inverse(e, totient):
    for d in range(3, totient): # nilai d dimulai dari 3 sampai totient n karena nilai d tidak boleh dibagi oleh totient n dan e tidak boleh dibagi oleh totient n
        if (d * e) % totient == 1: # (d * e) mod totient == 1 maka d adalah nilai yang dicari untuk private key (d)
            return d
    raise ValueError('Tidak ditemukan mod invers untuk e: %d, totient: %d' % (e, totient))

# Fungsi untuk membuat kunci publik dan kunci privat
def make_key():
    p = get_prime_input("Masukkan bilangan prima (p): ")
    q = get_prime_input("Masukkan bilangan prima (q): ")

    # p dan q tidak boleh sama
    while p == q:
        print("p dan q tidak boleh sama.")
        break

    # nilai n adalah hasil perkalian p dan q dan boleh dibagikan
    n = p * q
    # nilai totient n adalah hasil perkalian p-1 dan q-1 tidak boleh dibagikan karena untuk mencari nilai d
    totient_n = (p - 1) * (q - 1)

    # nilai e adalah nilai yang tidak boleh dibagikan dan harus lebih besar dari 2 dan lebih kecil dari Pembagi persekutuan terbesar / GCD
    e = int(input(f"Masukkan kunci publik e lebih dari 2 < e < {totient_n} and pembagi persekutuan terbesar(e, {totient_n}) = 1: "))


    while not (1 < e < totient_n and math.gcd(e, totient_n) == 1):
        print(f"Nilai tidak valid untuk e.")
        break

    d = mod_inverse(e, totient_n)

    print("kunci publik (e, n): ", e, n)
    print("kunci private (d, n): ", d, n)

def encryption_menu():
    e = int(input("Masukan kunci private e: "))
    n = int(input("Masukan nilai n: "))

    path = input("Masukkan path file: ")

    message = file_to_base64(path)
    md5message = hashlib.md5(message.encode())
    md5message = md5message.hexdigest()

    print(md5message)

    # Mengubah pesan menjadi nilai ASCII dan enkripsi
    ciphertext = ""

    for i in range(0, len(md5message)):
        block = md5message[i]
        encoded_block = ord(block)  # Merubah char menjadi ASCII desimal
        encrypted_block = pow(encoded_block, e, n)  # encoded_block^e mod n

        print(f"Block '{block}' is {encoded_block}, {encoded_block}^{e} mod {n} is {encrypted_block}")
        ciphertext += str(encrypted_block).zfill(3)  # Pastikan sudah 3 digit kalau belum tambahkan nilai padding 0

    print("Digital Signature : ", ciphertext)


def decryption_menu():
    path = input("Masukkan path file: ")

    message = file_to_base64(path)
    md5message = hashlib.md5(message.encode())
    md5message = md5message.hexdigest()

    d = int(input("Masukan kunci public d: "))
    n = int(input("Masukan nilai n: "))

    ciphertext_str = input("Masukkan Digital Signatures: ")
    ciphertext_blocks = [int(ciphertext_str[i:i+3]) for i in range(0, len(ciphertext_str), 3)]  # Pisahkan cipherteks menjadi blok 3 digit

    decrypted_message = ""

    for block in ciphertext_blocks:
        decrypted_block = pow(block, d, n) # block^d mod n
        decrypted_char = chr(decrypted_block)  # Merubah menjadi ascii kembali

        print(f"{block}^{d} mod {n} is {decrypted_block} which is '{decrypted_char}'")
        decrypted_message += decrypted_char

    print("\nHash Pesan : ", md5message)
    print("Digital Signature : ", decrypted_message)

    if (md5message == decrypted_message) :
        print("Pesan ini SAH.")
    else :
        print("Pesan ini Tidak SAH.")

def main():
    while True:
        print("\nMenu:")
        print("1. Membuat Kunci Publik dan Kunci Privat")
        print("2. Buat Digital Signature")
        print("3. Cek Digital Signature")
        print("4. Keluar Program")

        choice = input("Tentukan Pilihanmu (1, 2, 3, atau 4): ")
        if choice == '1':
            make_key()
        elif choice == '2':
            encryption_menu()
        elif choice == '3':
            decryption_menu()
        elif choice == '4':
            print("Terimakasih!")
            break
        else:
            print("Pilihanmu salah. Tolong masukkan 1, 2, 3, atau 4")

if __name__ == "__main__":
    main()


Menu:
1. Membuat Kunci Publik dan Kunci Privat
2. Buat Digital Signature
3. Cek Digital Signature
4. Keluar Program
Tentukan Pilihanmu (1, 2, 3, atau 4): 1
Masukkan bilangan prima (p): 19
Masukkan bilangan prima (q): 17
Masukkan kunci publik e lebih dari 2 < e < 288 and pembagi persekutuan terbesar(e, 288) = 1: 37
kunci publik (e, n):  37 323
kunci private (d, n):  109 323

Menu:
1. Membuat Kunci Publik dan Kunci Privat
2. Buat Digital Signature
3. Cek Digital Signature
4. Keluar Program
Tentukan Pilihanmu (1, 2, 3, atau 4): 2
Masukan kunci private e: 109
Masukan nilai n: 323
Masukkan path file: inputquiz.jpg
d79cfccaa59126018d7d6d432d105036
Block 'd' is 100, 100^109 mod 323 is 138
Block '7' is 55, 55^109 mod 323 is 55
Block '9' is 57, 57^109 mod 323 is 95
Block 'c' is 99, 99^109 mod 323 is 175
Block 'f' is 102, 102^109 mod 323 is 102
Block 'c' is 99, 99^109 mod 323 is 175
Block 'c' is 99, 99^109 mod 323 is 175
Block 'a' is 97, 97^109 mod 323 is 116
Block 'a' is 97, 97^109 mod 323 is 